In [1]:
#Imports with all of my tools
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
import random
from sympy import mod_inverse
from PIL import Image as im
%matplotlib inline

def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print(image.shape)
    return image

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
path = r"C:\Users\soule\OneDrive\Pictures\addie.jpg"
l = get_image(path)
img = im.open(path)
colors = img.getpixel((0, 1))
print(colors)
x, y, z = l.shape
# print(l)
data = im.fromarray(l.astype(np.uint8))
data.show()

(450, 600, 3)
(30, 31, 35)


<IPython.core.display.Javascript object>

In [4]:
def Generate_Point(a, b, p):
    x = random.randint(0, p)
    y = np.arange(0, p)
    possibilites = []
    for i in range(len(y)):
        yval = ((x ** 3) + (a * x) + b) % p
        # print(x[i],y[j],y[j]**2 % p, yval)
        # print(yval)
        if ((y[i]) ** 2) % p == yval:
            possibilites.append(y[i])
    if len(possibilites) == 0:
        return Generate_Point(a, b, p)
    else:
        yindex = random.randint(0, len(possibilites) - 1)
        return x, possibilites[yindex]

<IPython.core.display.Javascript object>

In [5]:
arr = []
for i in range(x):
    for j in range(y):
        for k in range(z):
            arr.append(l[i][j][k])
print(len(arr))

810000


<IPython.core.display.Javascript object>

In [6]:
totalPixs = x * y
print(totalPixs)

totalValues = x * y * z
print(totalValues)

270000
810000


<IPython.core.display.Javascript object>

In [7]:
imagePointsArray = np.reshape(arr, (totalValues // 2, 2), order="C")
print(imagePointsArray)

[[ 29  30]
 [ 34  30]
 [ 31  35]
 ...
 [135 137]
 [ 72 107]
 [110  43]]


<IPython.core.display.Javascript object>

In [8]:
p = 1046527  # 3803
a = -500
b = 761

<IPython.core.display.Javascript object>

In [9]:
# G = random point
Gx, Gy = Generate_Point(a, b, p)
print(Gx, Gy)

987000 593353


<IPython.core.display.Javascript object>

In [10]:
# Amanda's alpha and point A
private_a = 741
private_Ax, private_Ay = Generate_Point(a, b, p)
print(private_Ax, private_Ay)

559762 271120


<IPython.core.display.Javascript object>

In [11]:
# Bill's beta and point B
private_b = 9812
private_Bx, private_By = Generate_Point(a, b, p)
print(private_Bx, private_By)

524412 886655


<IPython.core.display.Javascript object>

In [12]:
def point_addition(x1, y1, x2, y2, p, a):
    z = 0
    if x1 == z and y1 == z:
        return x2, y2
    if x2 == z and y2 == z:
        return x1, y1
    if x1 == x2:
        if x1 == x2 and y1 == -y2 % p:
            return 0, 0
        else:
            den = int(mod_inverse(2 * y1, p) % p)
            m = int((3 * (x1 ** 2)) % p)
            m = int((m + a) % p)
            m = int((m * den) % p)
            m = int((((3 * (x1 ** 2)) % p + a) * (den)) % p)
            # m = (((3*(x1**2))%p + a) * (2*))
            # m = ((3*pow(x1, 2, p) + a)*pow(2*y1, p-2, p)) % p
            # print(m)
    else:
        den = int(mod_inverse(x1 - x2, p) % p)
        m = int((y1 - y2) % p)
        m = int((m * den) % p)
        # m = int(((y1 - y2)*den)%p)
        # print(m)
    x = int((m ** 2) % p)
    x = int((x - x1 - x2) % p)
    # x = int(((m**2)%p - x1 - x2) % p)
    # print(x)
    y = int((x1 - x) % p)
    y = int((y * m) % p)
    y = int((y - y1) % p)
    # y = int((m*(x1 - x)%p - y1) % p)
    # print(y)
    return int(x), int(y)


def point_subtraction(x1, y1, x2, y2, p, a):
    y2 = int(-(y2))
    y2 = int(y2 % p)
    return point_addition(x1, y1, x2, y2, p, a)


def point_multiplication(x, y, k, p, a):
    xnew, ynew = int(x), int(y)
    for i in range(k - 1):
        xnew, ynew = point_addition(x, y, xnew, ynew, p, a)
        # print(xnew,ynew)
    return int(xnew), int(ynew)

<IPython.core.display.Javascript object>

def point_addition(x1, y1, x2, y2, p, a):
    z = 0
    if x1 == z and y1 == z:
        return x2, y2
    if x2 == z and y2 == z:
        return x1, y1
    if x1 == x2:
        if x1 == x2 and y1 == -y2 % p:
            return 0, 0
        else:
            den = mod_inverse(2 * y1, p)
            m = (((3 * (x1 ** 2)) % p + a) * (den)) % p
    else:
        den = mod_inverse(x1 - x2, p)
        m = (y1 - y2) * den % p

    x = ((m ** 2) % p - x1 - x2) % p

    y = (m * (x1 - x) % p - y1) % p

    return x, y


def point_subtraction(x1, y1, x2, y2, p, a):
    y2 = -(y2)
    y2 = y2 % p
    return point_addition(x1, y1, x2, y2, p, a)


def point_multiplication(x, y, k, p, a):
    xnew, ynew = x, y
    for i in range(k - 1):
        xnew, ynew = point_addition(x, y, xnew, ynew, p, a)
        # print(xnew,ynew)
    return xnew, ynew

In [13]:
# G+A
G_plus_Ax, G_plus_Ay = point_addition(Gx, Gy, private_Ax, private_Ay, p, a)

<IPython.core.display.Javascript object>

In [14]:
# A_1 = alpha*(G+A)
A1x, A1y = point_multiplication(G_plus_Ax, G_plus_Ay, private_a, p, a)

<IPython.core.display.Javascript object>

In [15]:
# A_2 = alpha * A
A2x, A2y = point_multiplication(private_Ax, private_Ay, private_a, p, a)

<IPython.core.display.Javascript object>

In [16]:
# G+B
G_plus_Bx, G_plus_By = point_addition(Gx, Gy, private_Bx, private_By, p, a)

<IPython.core.display.Javascript object>

In [17]:
# B_1 = beta * (G+B)
B1x, B1y = point_multiplication(G_plus_Bx, G_plus_By, private_b, p, a)

<IPython.core.display.Javascript object>

In [18]:
# B_2 = beta * B
B2x, B2y = point_multiplication(private_Bx, private_By, private_b, p, a)

<IPython.core.display.Javascript object>

In [19]:
# A_B = alpha * B_2
A_Bx, A_By = point_multiplication(B2x, B2y, private_a, p, a)

<IPython.core.display.Javascript object>

In [20]:
# B_A = beta * A_2
B_Ax, B_Ay = point_multiplication(A2x, A2y, private_b, p, a)

<IPython.core.display.Javascript object>

In [21]:
# (beta + 1) * A_1
b_plus_one_A1 = point_multiplication(A1x, A1y, (private_b + 1), p, a)

# (beta + 1)*A_1 - A_2
b_plus_one_A1_minus_A2 = point_subtraction(
    b_plus_one_A1[0], b_plus_one_A1[1], A2x, A2y, p, a
)

# (beta + 1)*A_1 - A_2 + A_B
added_to_M = point_addition(
    b_plus_one_A1_minus_A2[0], b_plus_one_A1_minus_A2[1], A_Bx, A_By, p, a
)

<IPython.core.display.Javascript object>

In [22]:
coordinates, variables = imagePointsArray.shape
encrypted_array = []
remainder_array = []
print(coordinates)
r = coordinates
for i in range(coordinates):
    # M + (beta + 1)*A_1 - A_2 + A_B
    EP = point_addition(
        imagePointsArray[i][0],
        imagePointsArray[i][1],
        added_to_M[0],
        added_to_M[1],
        p,
        a,
    )
    encrypted_array.append(EP[0])
    encrypted_array.append(EP[1])
print(len(encrypted_array))

405000
810000


<IPython.core.display.Javascript object>

In [23]:
final_encrypted_array = []
for i in range(len(encrypted_array)):
    scaled_value = encrypted_array[i] % 256
    divisor = encrypted_array[i] // 256
    final_encrypted_array.append(scaled_value)
    final_encrypted_array.append(divisor)

<IPython.core.display.Javascript object>

In [24]:
encryption_arr = np.array(final_encrypted_array, dtype=int)
encryption_arr = np.reshape(encryption_arr, (2 * x, y, 3), order="C")

<IPython.core.display.Javascript object>

In [25]:
e_data = im.fromarray(encryption_arr.astype(np.uint8))
e_data.show()

<IPython.core.display.Javascript object>

In [26]:
# Decryption
data = np.asarray(e_data)
x, y, z = data.shape
print(x, y, z)
print(data.shape)
# print(data)

900 600 3
(900, 600, 3)


<IPython.core.display.Javascript object>

In [27]:
arr = []
for i in range(x):
    for j in range(y):
        for k in range(z):
            arr.append(data[i][j][k])
print(len(arr))
print(arr[0], arr[1], arr[2], arr[3])
# print(arr)

1620000
214 87 168 19


<IPython.core.display.Javascript object>

In [28]:
totalPixs = (x // 2) * y
print(totalPixs)
totalValues = x * y * z
print(totalValues)

270000
1620000


<IPython.core.display.Javascript object>

In [29]:
apply_divisor = np.reshape(arr, (totalValues // 2, 2), order="C")
applied_data = []
for i in range(totalValues // 2):
    newValue = apply_divisor[i][1] * 256 + apply_divisor[i][0]
    applied_data.append(newValue)
print(len(applied_data))

810000


<IPython.core.display.Javascript object>

In [30]:
x = x // 2

<IPython.core.display.Javascript object>

In [31]:
newTotal = x * y * z
EmPointsArray = np.reshape(applied_data, (newTotal // 2, 2), order="C")
print(EmPointsArray)

[[22486  5032]
 [37757 59283]
 [ 6378 32057]
 ...
 [36285   261]
 [19764 50280]
 [56400 64693]]


<IPython.core.display.Javascript object>

In [32]:
# G + B_1
G_plus_B1 = point_addition(Gx, Gy, B1x, B1y, p, a)

# alpha * (G+B_1) = alpha*G + alpha*B_1
private_a_G_plus_B1 = point_multiplication(G_plus_B1[0], G_plus_B1[1], private_a, p, a)

# alpha * (G+B_1) + B_A
private_a_G_plus_B1_plus_BA = point_addition(
    private_a_G_plus_B1[0], private_a_G_plus_B1[1], B_Ax, B_Ay, p, a
)

<IPython.core.display.Javascript object>

In [33]:
coordinates, variables = EmPointsArray.shape
decrypted_array = []
print(coordinates)
r=coordinates
for i in range(coordinates):
    #C_E - (alpha * (G+B_1) + B_A)
    DP = point_subtraction(EmPointsArray[i][0],EmPointsArray[i][1],private_a_G_plus_B1_plus_BA[0],private_a_G_plus_B1_plus_BA[1],p,a)
    decrypted_array.append(DP[0])
    decrypted_array.append(DP[1])


405000


<IPython.core.display.Javascript object>

In [34]:
decryption_arr = np.array(decrypted_array, dtype=int)
decryption_arr = np.reshape(decryption_arr, (x, y, z), order="C")
decryption_arr

array([[[ 342303,  951156,   99759],
        [  50820,  520054,  828210],
        [ 194345,  775927, 1013484],
        ...,
        [ 745624,  649264,  974740],
        [ 689669,  448882,  402111],
        [ 915628,  253697,   40576]],

       [[ 428199,  404642,  746686],
        [ 819646,  239325,  572714],
        [  23239,  138522,  654611],
        ...,
        [ 770832,   40057,   67874],
        [ 800146,   37484,  644708],
        [ 301009,  407311,  580489]],

       [[ 428199,  404642,  746686],
        [ 819646,  239325,  572714],
        [  23239,  138522,  654611],
        ...,
        [ 525605,  656665,  746289],
        [ 755155,  252336,  464843],
        [ 449206,  843709,  718971]],

       ...,

       [[ 387356,  238385,  329644],
        [ 376871,  794019,  705064],
        [ 680458,  405888,  671311],
        ...,
        [ 583443,   34070,  809433],
        [  13097,  626617,  692317],
        [ 272276,  596232,  544581]],

       [[ 338900,  640758, 1024152],
  

<IPython.core.display.Javascript object>

In [35]:
d_data = im.fromarray(decryption_arr.astype(np.uint8))
d_data.show()

<IPython.core.display.Javascript object>